In [2]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver


In [85]:
import re
import pandas as pd
import numpy as np
from urllib.parse import quote

df = pd.read_excel('BDP_Smiles_5.xlsx')
df_id = df.iloc[:,16]

driver = webdriver.Chrome('/Users/mariaclaraotani/chromedriver')
def lookup(link):
    t = quote(link)
    URL="https://zinc.docking.org/substances/subsets/for-sale/?q="+t
    driver.get(URL)
    link = driver.find_element_by_xpath('/html/body/div/div/div/nav[1]/div/div/div[4]/div[3]/form/button')
    link.click()
    elems = driver.find_elements_by_partial_link_text('ZINC')
    sString = "substance"
    results = []
    for elem in elems:
        link = elem.get_attribute("href")
        if sString in str(link):
            results.insert(0,link)
    return results

result = pd.Series([lookup(df_id[i]) for i in range (len(df_id))])
results = pd.DataFrame({'ZINC': result.values.tolist()})
df = pd.merge(df, results, right_index=True, left_index=True)
exploded = df.explode('ZINC')
exploded.to_excel('BDP_testttt.xlsx')



0    [https://zinc.docking.org/substances/ZINC00005...
1    [https://zinc.docking.org/substances/ZINC00010...
2    [https://zinc.docking.org/substances/ZINC00001...
dtype: object
   Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1   csdid      a     b      c  \
0           0       16140.0         16140.0  BINHUC   9.20  9.20  16.73   
0           0       16140.0         16140.0  BINHUC   9.20  9.20  16.73   
0           0       16140.0         16140.0  BINHUC   9.20  9.20  16.73   
0           0       16140.0         16140.0  BINHUC   9.20  9.20  16.73   
0           0       16140.0         16140.0  BINHUC   9.20  9.20  16.73   
1           1       20425.0         20425.0  BUFNEY  13.62  9.21  19.14   
1           1       20425.0         20425.0  BUFNEY  13.62  9.21  19.14   
2           1           NaN             NaN  BUFNEY    NaN   NaN    NaN   
2           1           NaN             NaN  BUFNEY    NaN   NaN    NaN   

   alpha    beta  gamma      Mw    Z         m  plane  low energy plan

In [79]:
import re
import pandas as pd
import numpy as np
from urllib.parse import quote

df = pd.read_excel('BDP_500_ZINC_MATCHES.xlsx')
df_id = df.iloc[:,18]
def consult(link):
    # link = "https://zinc.docking.org/substances/ZINC000012358719/"
    r = requests.get(link)
    soup = bs(r.content)
    counter = 0
    formula = ""
    mwt = 0
    for table_row in soup.select("table tr"):
        cells = table_row.findAll("td")
        if len(cells) > 0:
            if counter == 0:
                mwt = float(cells[3].text.strip())
                counter+=1
            elif counter == 1:
                formula = cells[0].text.strip()
                counter+=1
            else:
                break
    mydivs = soup.find("div", {"class": "panel-heading clearfix"})
    a = mydivs.find("a", recursive=False).text.strip()
    item = a.replace(' Items Total', '')
    hits = int(item)
    return formula, mwt, hits

result = pd.Series([consult(df_id[i]) for i in range (len(df_id))])
results = pd.DataFrame(result.values.tolist(), index = result.index, columns = ['formula', 'mwt','hits'])
df = pd.merge(df, results, right_index=True, left_index=True)

df.to_excel('BDP_ZINC_FINALIZADO.xlsx')


0     (C19H19NO4, 325.364, 7)
1     (C19H19NO4, 325.364, 8)
2     (C19H19NO4, 325.364, 7)
3    (C19H19NO4, 325.364, 34)
4     (C22H17N2P, 340.366, 4)
5    (C9H11NO2, 165.192, 140)
dtype: object
